🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys.

# Building Advanced RAG Systems with Haystack SuperComponents

This notebook demonstrates how to build an advanced Retrieval-Augmented Generation (RAG) system using Haystack's SuperComponent feature. We'll cover:

1. Setting up a hybrid RAG pipeline with both dense and sparse retrieval
2. Creating a SuperComponent for simplified interface
3. Building tools from components for agent-based systems
4. Implementing a multi-tool agent for complex queries

## Prerequisites
- Basic understanding of RAG systems
- Familiarity with Haystack components
- OpenAI API key for LLM access
- SerperDev API key for web search capabilities

## Learning Objectives
By the end of this notebook, you will be able to:
- Build a hybrid RAG pipeline combining multiple retrieval methods
- Create a SuperComponent to simplify pipeline interfaces
- Transform components into tools for agent-based systems
- Implement an agent that can use multiple tools for complex queries

In [1]:
from scripts.indexing import document_store
from dotenv import load_dotenv
load_dotenv(".env")

/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch4/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running unified indexing pipeline for web, local files, and CSV...


Error processing document 1384ec36dd6d99f90ab589732d5219b7371dac846d0f0bd89c6385189c4079c0. Keeping it, but skipping cleaning. Error: Error tokenizing data. C error: Expected 5 fields in line 5, saw 7

Error processing document 1384ec36dd6d99f90ab589732d5219b7371dac846d0f0bd89c6385189c4079c0. Keeping it, but skipping splitting. Error: Error tokenizing data. C error: Expected 5 fields in line 5, saw 7

Batches: 100%|██████████| 5/5 [00:00<00:00,  6.59it/s]


True

## 1. Setting Up Components

In this section, we'll set up the core components needed for our hybrid RAG pipeline. We'll use both dense and sparse retrieval methods to achieve better search results:

1. **Dense Retrieval**: Uses embeddings to find semantically similar documents
   - SentenceTransformersTextEmbedder: Converts text into vector representations
   - InMemoryEmbeddingRetriever: Searches for similar vectors in the document store

2. **Sparse Retrieval**: Uses keyword matching (BM25 algorithm)
   - InMemoryBM25Retriever: Performs traditional keyword-based search

3. **Post-processing**:
   - DocumentJoiner: Combines results from both retrievers
   - SentenceTransformersSimilarityRanker: Re-ranks results for better precision

4. **Generation**:
   - PromptBuilder: Creates structured prompts with context
   - OpenAIGenerator: Generates responses using GPT model
   - OllamaGenerator: Generates responses using Ollama

In [2]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
from haystack import Pipeline

from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.joiners import DocumentJoiner
from haystack.components.rankers import SentenceTransformersSimilarityRanker

# --- 1. Initialize Query Pipeline Components ---

# Text Embedder: To embed the user's query. Must be compatible with the document embedder.
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

# Retriever: Fetches documents from the DocumentStore based on vector similarity.
retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=3)

# PromptBuilder: Creates a prompt using the retrieved documents and the query.
# The Jinja2 template iterates through the documents and adds their content to the prompt.
prompt_template_for_pipeline = """
Given the following information, answer the user's question.
If the information is not available in the provided documents, 
say that you don't have enough information to answer.

Context:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
prompt_builder_inst = PromptBuilder(template=prompt_template_for_pipeline,
                                    required_variables="*")
llm_generator_inst = OpenAIGenerator(api_key=Secret.from_env_var("OPENAI_API_KEY"), model="gpt-4o-mini")


# IF YOU PREFER TO USE OLLAMA INSTEAD OF OPENAI, UNCOMMENT THE FOLLOWING LINES AND COMMENT OUT THE OPENAI GENERATOR ABOVE
# downlod Ollama https://ollama.com/download and install it
# run in your terminal
# $ ollama pull mistral-nemo:12b

# then uncomment below
# from haystack_integrations.components.generators.ollama import OllamaGenerator

# llm_generator_inst = OllamaGenerator(model="mistral-nemo:12b",
#                             generation_kwargs={
#                               "num_predict": 100,
#                               "temperature": 0.9,
#                               })



# Sparse Retriever (BM25): For keyword-based search.
# This retriever needs to be "warmed up" by calculating statistics on the documents in the store.
bm25_retriever = InMemoryBM25Retriever(document_store=document_store, top_k=3)

# DocumentJoiner: To merge the results from the two retrievers.
# The default 'concatenate' mode works well here as the ranker will handle final ordering.
document_joiner = DocumentJoiner()

# Ranker: A cross-encoder model to re-rank the combined results for higher precision.
# This model is highly effective at identifying the most relevant documents from a candidate set.
ranker = SentenceTransformersSimilarityRanker(model="BAAI/bge-reranker-base", top_k=3)



## 2. Building the Hybrid RAG Pipeline

A hybrid RAG (Retrieval Augmented Generation) pipeline combines multiple retrieval methods to improve the quality of document search. Here's how we'll build it:

1. **Component Creation**:
   - Initialize both dense and sparse retrievers
   - Set up the document joiner and ranker
   - Configure the prompt builder and generator

2. **Pipeline Assembly**:
   - Chain components together in a logical sequence
   - Define how documents flow through the pipeline
   - Set parameters for each component

3. **Benefits**:
   - Better search accuracy by combining methods
   - More robust to different types of queries
   - Improved context selection for generation

The resulting pipeline will provide both semantic understanding and keyword matching capabilities.

In [3]:
# --- 2. Build the Hybrid RAG Pipeline ---
hybrid_rag_pipeline = Pipeline()

# Add all necessary components
hybrid_rag_pipeline.add_component("text_embedder", text_embedder)
hybrid_rag_pipeline.add_component("embedding_retriever", retriever) # Dense retriever
hybrid_rag_pipeline.add_component("bm25_retriever", bm25_retriever) # Sparse retriever
hybrid_rag_pipeline.add_component("document_joiner", document_joiner)
hybrid_rag_pipeline.add_component("ranker", ranker)
hybrid_rag_pipeline.add_component("prompt_builder", prompt_builder_inst)
hybrid_rag_pipeline.add_component("llm", llm_generator_inst)

In [4]:
# --- 3. Connect the Components in a Graph ---

# The query is embedded for the dense retriever
hybrid_rag_pipeline.connect("text_embedder.embedding", "embedding_retriever.query_embedding")

# The raw query text is sent to the BM25 retriever and the ranker
# Note: The query input for these components is the raw text string.

# The outputs of both retrievers are fed into the document joiner
hybrid_rag_pipeline.connect("embedding_retriever.documents", "document_joiner.documents")
hybrid_rag_pipeline.connect("bm25_retriever.documents", "document_joiner.documents")

# The joined documents are sent to the ranker
hybrid_rag_pipeline.connect("document_joiner.documents", "ranker.documents")

# The ranked documents are sent to the prompt builder
hybrid_rag_pipeline.connect("ranker.documents", "prompt_builder.documents")

# The final prompt is sent to the LLM
hybrid_rag_pipeline.connect("prompt_builder.prompt", "llm.prompt")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - embedding_retriever: InMemoryEmbeddingRetriever
  - bm25_retriever: InMemoryBM25Retriever
  - document_joiner: DocumentJoiner
  - ranker: SentenceTransformersSimilarityRanker
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> embedding_retriever.query_embedding (list[float])
  - embedding_retriever.documents -> document_joiner.documents (list[Document])
  - bm25_retriever.documents -> document_joiner.documents (list[Document])
  - document_joiner.documents -> ranker.documents (list[Document])
  - ranker.documents -> prompt_builder.documents (list[Document])
  - prompt_builder.prompt -> llm.prompt (str)

## 3. Creating the SuperComponent

A SuperComponent is a special type of pipeline component that can contain and coordinate multiple sub-pipelines. Here's what makes it powerful:

1. **Encapsulation**:
   - Groups related components into a single unit
   - Manages internal data flow and state
   - Provides a clean interface to the outside

2. **Flexibility**:
   - Can switch between different sub-pipelines
   - Adapts behavior based on input or conditions
   - Easy to modify internal logic

3. **Reusability**:
   - Package complex behavior into a single component
   - Share across different pipelines
   - Maintain consistency in processing

The SuperComponent pattern helps manage complexity while keeping our pipeline modular and maintainable.

In [5]:
# Create a super component with simplified input/output mapping
from haystack import SuperComponent

hybrid_rag_sc = SuperComponent(
    pipeline=hybrid_rag_pipeline,
    input_mapping={
        "query": ["text_embedder.text", 
                  "bm25_retriever.query",
                  "ranker.query",
                  "prompt_builder.question"],
    },
    output_mapping={
        "llm.replies": "replies",
        "ranker.documents": "documents"
    }
)


In [6]:
# Run the pipeline with simplified interface
no_answer_question = hybrid_rag_sc.run(query="What is the capital of France?")

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.75it/s]


In [7]:
import json
from IPython.display import Markdown, display

def pretty_print_response(response):
    # Display as Markdown for better formatting
    display(Markdown(response))

In [8]:
pretty_print_response(no_answer_question['replies'][0])

I don't have enough information to answer.

In [9]:
pdf_ai_question = hybrid_rag_sc.run(query="Summarize how people use AI?")

Batches: 100%|██████████| 1/1 [00:00<00:00, 21.05it/s]


In [10]:
pretty_print_response(pdf_ai_question['replies'][0])

People use AI, particularly generative AI, in various ways both at work and outside of work. The primary usage includes seeking information or advice to make better decisions, acting as an advisor or research assistant, rather than just performing specific job tasks. This usage is classified into three intents: Asking (seeking information or advice), Doing (performing tasks or actions), and Expressing (communicating feelings or ideas). Generative AI improves worker output by providing decision support, especially in knowledge-intensive jobs, where the quality of decision-making enhances productivity.

## 5. Creating Tools from Components

Components can be transformed into tools that agents can use. Here's how:

1. **Tool Creation**:
   - Define tool interface and parameters
   - Map component functionality to tool actions
   - Add input validation and error handling

2. **Tool Configuration**:
   - Set default parameters
   - Define input/output formats
   - Add usage documentation

3. **Integration with Agents**:
   - Register tools with agent
   - Define tool selection logic
   - Handle tool responses

This abstraction allows agents to use complex pipeline functionality through a simple interface.

In [11]:

from haystack.tools.component_tool import ComponentTool

# --- 1. Create a Tool from our Supercomponent ---

# The name should be a simple, machine-readable identifier.
tool_name = "internal_document_search"

# The description is crucial. It tells the agent's LLM what the tool is for.
# It should be clear, detailed, and specific.
tool_description = (
    "Use this tool to search and answer questions about internal knowledge, "
    "including information about Haystack, LLM models, and AI frameworks. "
    "This is the primary source for any questions related to company-specific data."
)

# Wrap the supercomponent instance in a ComponentTool
internal_search_tool = ComponentTool(
    name=tool_name,
    component=hybrid_rag_sc,
    description=tool_description,
)

print("RAG Supercomponent has been successfully wrapped into a Tool.")

RAG Supercomponent has been successfully wrapped into a Tool.


In [12]:
# Set the SERPER_API_KEY environment variable.

from haystack.components.agents import Agent
from haystack.components.websearch import SearchApiWebSearch
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator


# --- 1. Create a Web Search Tool ---

# Instantiate the web search component
web_search_component = SearchApiWebSearch(api_key=Secret.from_env_var("SEARCHAPI_API_KEY"))

# Wrap it in a ComponentTool with a clear name and description
web_search_tool = ComponentTool(
    name="web_search",
    component=web_search_component,
    description="Use this tool to search the public internet for current events, news, and general knowledge. "
                "It is best for information that is not specific to our internal documents.",
)



## 6. Building an Agent with Multiple Tools

Agents become more powerful when equipped with multiple tools. Here's our approach:

1. **Agent Architecture**:
   - Define agent's capabilities and goals
   - Create tool selection strategy
   - Implement decision-making logic

2. **Tool Management**:
   - Register multiple tools
   - Handle tool dependencies
   - Manage tool state

3. **Coordination**:
   - Select appropriate tools for tasks
   - Chain tool operations
   - Handle tool failures

4. **Benefits**:
   - More flexible problem-solving
   - Better task completion rates
   - Adaptable to different scenarios

The resulting agent can handle complex tasks by combining tool capabilities.

In [13]:
# --- 2. Initialize the Agent ---

# The agent needs a list of all available tools
tools = [internal_search_tool, web_search_tool]

# The agent's reasoning is powered by an LLM. It must be a model that supports tool calling.
agent_llm = OpenAIChatGenerator(model="gpt-4o-mini")

# Alternatively, you can use an open-source chat model with Ollama
# from haystack_integrations.components.generators.ollama import OllamaGenerator, OllamaChatGenerator
# agent_llm = OllamaChatGenerator(model="mistral-nemo:12b",
#                             generation_kwargs={
#                               "num_predict": 100,
#                               "temperature": 0.9,
#                             })


# Define a system prompt to guide the agent's behavior
system_prompt = """
You are a helpful research assistant. Your goal is to answer the user's question accurately and comprehensively.
You have access to two tools:
1. internal_document_search: For questions about our internal knowledge base (Haystack, AI models, etc.).
2. web_search: For questions about current events or general public information.

First, think about which tool is most appropriate for the user's question.
Then, call that tool with the necessary query.
If the question requires information from both sources, you can call the tools sequentially.
Finally, synthesize the information from the tools into a final answer for the user.
"""

# Instantiate the Agent
agent = Agent(chat_generator=agent_llm, tools=tools, system_prompt=system_prompt)

## 7. Running Complex Queries

Let's explore how our agent handles complex queries using multiple tools:

1. **Query Processing**:
   - Parse user input
   - Identify required tools
   - Plan execution strategy

2. **Tool Orchestration**:
   - Execute tools in sequence
   - Handle intermediate results
   - Combine tool outputs

3. **Result Generation**:
   - Synthesize final response
   - Format output
   - Provide explanations

We'll demonstrate this with increasingly complex query examples to show the system's capabilities.

In [14]:
# --- 3. Run the Agent with a Complex Query ---

# This query requires both internal knowledge (about Haystack) and external knowledge (current news).
complex_query = (
    "Using the internal documents, explain how people use AI, then investigate the latest trends in 2025 in AI from a web search."
)

print(f"\nRunning agent with complex query: '{complex_query}'")

# The agent will now perform a multi-step reasoning process.
# We can inspect the 'transcript' to see its thoughts and actions.
agent_result = agent.run(messages=[ChatMessage.from_user(complex_query)])



Running agent with complex query: 'Using the internal documents, explain how people use AI, then investigate the latest trends in 2025 in AI from a web search.'


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.88it/s]


In [15]:
def pretty_print_agent_steps(agent_result):
    """
    Pretty prints the steps taken by the agent and its final response.
    
    Args:
        agent_result (dict): The result dictionary from the agent's run
    """
    # Print system message
    system_msg = agent_result['messages'][0]._content[0].text
    display(Markdown("## System Prompt"))
    display(Markdown(f"```\n{system_msg}\n```"))
    
    # Print user query
    user_query = agent_result['messages'][1]._content[0].text
    display(Markdown("## User Query"))
    display(Markdown(f"*{user_query}*"))
    
    # Print tool calls and results
    display(Markdown("## Agent Actions"))
    for msg in agent_result['messages']:
        if msg._role.value == 'assistant' and hasattr(msg._content[0], 'tool_name'):
            for tool_call in msg._content:
                display(Markdown(f"**Tool Used:** {tool_call.tool_name}"))
                display(Markdown(f"**Query:** {tool_call.arguments['query']}"))
    
    # Print final response
    display(Markdown("## Final Response"))
    display(Markdown(agent_result['last_message']._content[0].text))

# Use the function
pretty_print_agent_steps(agent_result)

## System Prompt

```

You are a helpful research assistant. Your goal is to answer the user's question accurately and comprehensively.
You have access to two tools:
1. internal_document_search: For questions about our internal knowledge base (Haystack, AI models, etc.).
2. web_search: For questions about current events or general public information.

First, think about which tool is most appropriate for the user's question.
Then, call that tool with the necessary query.
If the question requires information from both sources, you can call the tools sequentially.
Finally, synthesize the information from the tools into a final answer for the user.

```

## User Query

*Using the internal documents, explain how people use AI, then investigate the latest trends in 2025 in AI from a web search.*

## Agent Actions

...

**Query:** how people use AI

**Tool Used:** web_search

**Query:** latest trends in AI 2025

## Final Response

### How People Use AI

Based on our internal documents, AI is being used in a variety of sectors and for many purposes, broadly categorized as follows:

1. **Professional Use**:
   - AI is significantly augmenting workplace tasks, leading to enhanced productivity in roles such as computer programming and IT support. Approximately 37% of AI interactions are related to 'computer and mathematical' occupations, indicating a strong usage in technical fields.
   - The workplace applications of AI include automating routine tasks, assisting in software development, and even facilitating complex decision-making processes.

2. **Generative AI**:
   - This technology is versatile, enabling users to seek practical guidance, create content, and even engage in therapeutic interactions. For instance, many users consult AI for creating customized workout plans or seeking informational advice tailored to their specific needs.
   - The adoption of generative AI spans across different demographics, indicating a widening user base with increasingly varied applications.

3. **Social Interactions**:
   - Beyond professional contexts, AI is being utilized for social purposes, particularly in mental health applications where AI serves as a companion or therapeutic assistant. Interestingly, this is reported to be one of the most prevalent use cases for generative AI.

4. **User Intent Classification**:
   - Users approach AI with different intents—ranging from asking specific questions, seeking help with tasks (Doing), or expressing feelings and thoughts (Expressing). Each interaction type reflects a unique way people leverage AI for their personal and professional growth.

### Latest Trends in AI (2025)

A web search reveals several key trends shaping the AI landscape in 2025:

1. **Integration and Autonomy**:
   - AI technologies are becoming increasingly integrated into daily life, facilitating not only work but also personal tasks. AI-powered agents are expected to display greater autonomy, simplifying complex tasks for users.

2. **Generative AI Surge**:
   - The trend towards generative AI continues, enhancing areas like entertainment, healthcare, and education. Its impact is anticipated to be profound as it transforms creative processes and interactions.

3. **Multimodal AI**:
   - There’s a growing emphasis on multimodal AI capabilities, which combine various forms of input (text, sound, images) enhancing contextual understanding and the effectiveness of AI applications.

4. **AI in Public Sector**:
   - AI applications in the public sector are increasing, driving efficiency in government and social services. This involves the use of AI for data analysis, predictive modeling, and improving citizen engagement.

5. **Open-Weight Models**:
   - The gap between closed and open-weight AI models is narrowing, leading to more democratized AI technologies that are accessible and affordable.

6. **Ethical Considerations and Regulations**:
   - With the potent capabilities of AI growing, there is an increasing focus on ethical implications and the importance of regulatory frameworks to guide its development and use, ensuring responsible implementation.

Overall, the interplay between the evolving uses of AI and emerging trends highlights a dynamic landscape where technology continues to adapt and integrate more seamlessly into various aspects of life and work. For further reading on specific trends, you can access the following resources:
- [The 2025 AI Index Report](https://hai.stanford.edu/ai-index/2025-ai-index-report)
- [The State of AI: Global Survey 2025](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai)
- [6 AI Trends You'll See More Of In 2025](https://news.microsoft.com/source/features/ai/6-ai-trends-youll-see-more-of-in-2025/)